<a href="https://colab.research.google.com/github/BeautifulTovarisch/statistics-potpourri/blob/main/basic_probability/conditional_probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Indepdendence and Conditional Probability

Two events are said to be **independent** when their outcomes do not influence one another's (e.g a coin toss).

> NOTE: This is _not_ the same as **disjoint**!

The probability of two idependent events is give by straightforward multiplication. Assume $A$ and $B$ are independent events in sample space $\Omega$. Then

\begin{equation}
P(A \cap B) = P(A)P(B)
\end{equation}

## Conditional Probability

When an event $B$ reveals some information about the probability of some other event $A$, we compute the probability of $A$ given the _condition_ of $B$ having occurred:

\begin{equation}
P(A \mid B) = \frac {P(A \cap B)} {P(B)}
\end{equation}


> The Probability of $A$ given $B$

Immediately from the previous formula we see that if $A$ and $B$ are independent, $P(A \mid B) = P(A)$.

### Example

Here is a canonical example with fair dice. Let $A$ be the event that the sum of dice rolls is even and let $B$ be the event that at least one of the dice lands on a two.

\begin{equation}
  P(2 \mid \text{even})
    = \frac {P(2 \cap \text{even})} {P(\text{even})}
    = \frac {5/36} {18/36}
    = \frac 5 {18}
\end{equation}

## Law of Total Probability

Rearranging the formula for conditional probability, we arrive at:

\begin{equation}
  P(A \cap B) = P(B)P(A \mid B)
\end{equation}

which can be used to compute of the outcomes formed from a disjoint union of an event $A$ and a _partition_ $\{B_1, B_2, \dots, B_n\}$ over $\Omega$:

\begin{equation}
  P(A)
    = \sum_{k=0}^{\infty} P(A \cap B_k)
    = \sum_{k=0}^{\infty} P(A \mid B_k)P(B_k)
\end{equation}

which we call the **Law of Total Probability**.

In [ ]:
import Pkg

Pkg.add("SpecialFunctions")

## Manufacturing Defects

As an example of the law of total probability, consider the probability of a defect in a semi-conductor manufacturing process. In this example, let $A$ be the event of a failure and assume each $B_k$ forms a partition over $\Omega$ where $k$ is the number of dust particles present in the room.

We assume a model for failures given the number of particles present:

\begin{array}{cc}
  P(A \mid B_k) = 1 - \frac 1 {k + 1}
  & P(B_k) = \frac 6 {\pi^2 (k + 1)^2}
\end{array}

(somehow) this works out to $\sum_{k=0}^\infty k^{-2} = \frac {\pi^2} 6$, which (somehow) implies that $\sum_{k=0}^\infty P(B_k) = 1$.

Because we have each $B_k$ forming a partition, we can use the formula for total probability as follows to compute the probability of a manufacturing failure:

\begin{align}
  P(A) &= \sum_{k=0}^\infty P(A \mid B_k) P(B_k) \\
  &= \sum_{k=0}^\infty \left(1 - \frac 1 {k + 1}\right) \frac 6 {\pi^2 (k+1)^2}
\end{align}

which (somehow) evaluates to

\begin{equation}
  P(A) = 1 - \frac {6 \xi(3)} {\pi^2} \approx 0.2692
\end{equation}

where $\xi(s) = \sum_{n=1}^\infty \frac 1 {n^s}$, which is called the _Riemann Zeta Function_.

In [3]:
"""
Manufacturing Defects

This simulation computes `P(A)` above by evaluating the summation and via closed
form using the Riemann Zeta function.
"""

using SpecialFunctions

# Evidently this is enough to produce an accurate enough convergence.
n = 2000

pB(k) = 6 / (pi^2 * (k + 1)^2)
pBGivenA(k) = 1 - 1/(k+1)

numerical = sum([pB(k) * pBGivenA(k) for k in 0:n])
analytical = 1 - 6*zeta(3)/pi^2

println("numerical = $numerical\t analytical = $analytical")

numerical = 0.26893337073278945	 analytical = 0.26923703059856086


## Bayes Theorem

Manipulating the formula for conditional probability we can arrive at an important theorem.

Notice $P(B \mid A) = \frac {P(A \cap B)} {P(A)} \implies P(A \cap B) = P(A)P(B \mid A)$. Applying this transformation to the standard formula:

\begin{equation}
  P(A \mid B)
    = \frac {P(A \cap B)} {P(B)}
    = \frac {P(A)P(B \mid A)} {P(B)}
\end{equation}

This enables us a way to compute the probability of a _prior outcome_ given the observation of a _posterior condition_. In other words, we can proceed in the reverse direction of a conditional probability.

### Monty Hall Problem

The Monty Hall problem invariably shows up during any introduction to conditional probability due to its unintuitive conclusion. In the original gameshow _Let's Make a Deal_, the host prompted the guest to choose among three doors. Behind one door was a prize, and behind the other two were goats. After the guest made their initial selection, the host would reveal one of the doors containing a goat and give the player the opportunity to switch their answer.

> Note: I prefer to brute-force count this problem, but I've managed to do this with k-combinations. I'll add that solution here some other time.

#### Analysis

For simplicity, assume the player chooses Door 1 in the following scenarios.

|Scenario|Action|Result|
|--------|------|------|
|CGG     |Stay  |W     |
|GCG     |Stay  |L     |
|GGC     |Stay  |L     |
|CGG     |Switch|L     |
|GCG     |Switch|W     |
|GGC     |Switch|W     |

From the table above, we can see that the heuristic to switch actually has a better probability than sticking with the original choice. This is because the rules of the game force the host to eliminate one of the bad choices, no matter what the initial selection was.

Analytically, we can use conditional probability to compare the strategies. Let $A_i$ be the event that door $i$ reveals the prize and $B_i$ be the event that the host reveals door $i$.

The first strategy:

\begin{equation}
  P(A_1 \mid B_2)
    = \frac {P(B_2 \mid A_1)P(A_1)} {P(B_2)}
    = \frac {1/2 * 1/3} {1/2}
    = \frac 1 3
\end{equation}

and the second in which the player switches to door 3 (assume without loss of generality that B2 is opened):

\begin{equation}
  P(A_3 \mid B_2)
    = \frac {P(B_2 \mid A_3) P(A_3)} {P(B_2)}
    = \frac {1 * 1/3} {1/2}
    = \frac 2 3
\end{equation}

Note that $P(B_2 \mid A_3) = 1$ because the host has no choice but to reveal the door without the prize.

In [17]:
"""
Monty Hall Simulation

This Monte Carlo simulation is a straightforward random experiment modeling the
famous Monty Hall problem. The probability of winning with a strategy of keeping
the original choice is compared to one in which the contestant always switches.
"""

using Random
Random.seed!(1)

# TODO: Almost certain this can be done with bits + XOR
function monty_hall(switch)
  prize, choice = rand(1:3), rand(1:3)

  if prize == choice
    # Choose randomly between the remaining doors
    rev = rand(setdiff(1:3, choice))
  else
    rev = rand(setdiff(1:3, [prize, choice]))
  end

  if switch
    choice = setdiff(1:3, [rev, choice])[1]
  end

  return choice == prize
end

N = 10^6
strat1 = sum([monty_hall(false) for _ in 1:N]) / N
strat2 = sum([monty_hall(true) for _ in 1:N]) / N

println("Don't Switch = $strat1 \t Switch = $strat2")

Don't Switch = 0.333636 	 Switch = 0.666896
